In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download NLTK data files (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Sample corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "I love programming in Python",
    "Natural language processing is fascinating",
    "Word embeddings are useful in many NLP tasks",
    "CBOW is a great way to train word vectors"
]

# Preprocessing
stop_words = set(stopwords.words('english'))
tokenized_corpus = []
for sentence in corpus:
    tokens = word_tokenize(sentence.lower())
    filtered_tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    tokenized_corpus.append(filtered_tokens)

# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_corpus)
total_words = len(tokenizer.word_index) + 1  # Plus one for padding

# Create CBOW dataset
def create_cbow_dataset(corpus, window_size):
    context_words = []
    target_words = []

    for tokens in corpus:
        for i, target in enumerate(tokens):
            start = max(0, i - window_size)
            end = min(len(tokens), i + window_size + 1)
            context = [tokens[j] for j in range(start, end) if j != i]
            target_words.append(target)
            context_words.append(context)

    return context_words, target_words

# Set window size
window_size = 2
context, target = create_cbow_dataset(tokenized_corpus, window_size)

# Convert context and target words to sequences
context_sequences = tokenizer.texts_to_sequences([' '.join(c) for c in context])
target_sequences = tokenizer.texts_to_sequences(target)

# Pad the context sequences
max_length = max(len(seq) for seq in context_sequences)  # Find the maximum length
X = pad_sequences(context_sequences, maxlen=max_length, padding='post')
y = np.array(target_sequences).flatten()

# Convert target sequences to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the CBOW model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=total_words, output_dim=64, input_length=X.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Example context to test
test_context = "the brown fox".split()
test_sequence = tokenizer.texts_to_sequences([' '.join(test_context)])

# Pad the test sequence
test_sequence = pad_sequences(test_sequence, maxlen=max_length, padding='post')

# Make predictions
predicted = model.predict(test_sequence)
predicted_word_index = np.argmax(predicted, axis=1)

# Display the predicted word
predicted_word = tokenizer.index_word[predicted_word_index[0]]
print(f"Predicted Word for context '{' '.join(test_context)}': {predicted_word}")
